### Setup

In [7]:
using Pkg

Pkg.add(url="https://github.com/mppmu/BoostFractor.jl.git")
Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

    Updating git-repo `https://github.com/mppmu/BoostFractor.jl.git`


   Resolving package versions...


   Installed FFTW ───────────── v1.7.0


   Installed CommonSolve ────── v0.2.4
   Installed ConstructionBase ─ v1.5.2
   Installed LogExpFunctions ── v0.3.24
   Installed ChainRulesCore ─── v1.16.0


    Updating `C:\Users\domin\.julia\environments\v1.9\Project.toml`
  [222d2961] + BoostFractor v0.0.0 `https://github.com/mppmu/BoostFractor.jl.git#master`
    Updating `C:\Users\domin\.julia\environments\v1.9\Manifest.toml`
  [621f4979] + AbstractFFTs v1.3.1
  [222d2961] + BoostFractor v0.0.0 `https://github.com/mppmu/BoostFractor.jl.git#master`
  [d360d2e6] + ChainRulesCore v1.16.0
  [38540f10] + CommonSolve v0.2.4
  [34da2185] + Compat v4.6.1
  [187b0558] + ConstructionBase v1.5.2
  [ffbed154] + DocStringExtensions v0.9.3
  [7a1cc6ca] + FFTW v1.7.0
  [b21f74c0] + FunctionZeros v0.2.0
⌅ [92d709cd] + IrrationalConstants v0.1.1
  [692b3bcd] + JLLWrappers v1.4.1
  [2ab3a3ac] + LogExpFunctions v0.3.24
  [1914dd2f] + MacroTools v0.5.10
  [189a3867] + Reexport v1.2.2
  [ae029012] + Requires v1.3.0
⌅ [f2b01f46] + Roots v1.4.1
⌅ [efcf1570] + Setfield v0.8.2
⌅ [276daf66] + SpecialFunctions v1.8.8
  [f5851436] + FFTW_jll v3.3.10+0
  [1d5cc7b8] + IntelOpenMP_jll v2023.1.0+0
⌅ [856f044c] + MKL_

...

  ✓ Dragoon


  ✓ Plots → UnitfulExt
  110 dependencies successfully precompiled in 81 seconds. 43 already precompiled.


    Updating registry at `C:\Users\domin\.julia\registries\General.toml`
    Updating git-repo `https://github.com/mppmu/BoostFractor.jl.git`


    Updating git-repo `https://github.com/bergermann/Dragoon.jl.git`


  No Changes to `C:\Users\domin\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\domin\.julia\environments\v1.9\Manifest.toml`

In [8]:
using Dragoon

In [29]:
#number of discs in the booster
n = 20

#initial disc configuration
#findpeak tries to find an equidistant configuration with a peak at f
initdist = findpeak(22.025e9,n)

#generate frequencies for calculation and for plotting
freqs = genFreqs(22.025e9,50e6; length=10) #optimize on these frequencies
freqsplot = genFreqs(22.025e9,150e6; length=1000)

#initialize physical properties of the booster
booster = AnalyticalBooster(initdist)
#=
Booster(
    pos,              disc positions
    ndisk,            disc number
    thickness,        disc thickness
    epsilon,          disc epsilon
    vmotor,           motor speed
    maxlength,        maximum allowed booster length (not implemented yet)
    timestamp,        booster operation time
    summedtraveltime, summed motor operation time
    codetimestamp     code runtime
)
=#

#initialize storage of measurements
hist = initHist(booster,10000,freqs,ObjAnalytical)
#                           ^ length, should be at least the larger of (n^2, search steps)
;

Objective Value: -243.3, Timestamp: 2023-06-18T22:02:22.758


In [30]:
b1 = copy(booster.pos); display(b1)

20-element Vector{Float64}:
 0.007209715011297336
 0.01541943002259467
 0.023629145033892004
 0.03183886004518934
 0.04004857505648668
 0.04825829006778401
 0.05646800507908135
 0.06467772009037868
 0.07288743510167603
 0.08109715011297336
 0.08930686512427068
 0.097516580135568
 0.10572629514686534
 0.11393601015816267
 0.12214572516946
 0.13035544018075734
 0.13856515519205465
 0.14677487020335206
 0.15498458521464936
 0.1631943002259467

In [31]:
p = zeros(booster.ndisk)
g = zeros(booster.ndisk)
h = zeros(booster.ndisk,booster.ndisk)

updateHist!(booster,hist,freqs,ObjAnalytical)

deriv = Derivator2(1e-3,1e-3,"double")

secondDerivative(g,h,booster,hist,freqs,ObjAnalytical,(1e-3,1e-3,"double"))

b2 = copy(booster.pos); display(booster.pos)

20-element Vector{Float64}:
 0.007209715011297334
 0.01541943002259467
 0.023629145033892004
 0.03183886004518934
 0.04004857505648668
 0.04825829006778401
 0.05646800507908135
 0.06467772009037868
 0.07288743510167603
 0.08109715011297336
 0.08930686512427068
 0.097516580135568
 0.10572629514686534
 0.11393601015816267
 0.12214572516946
 0.13035544018075734
 0.13856515519205465
 0.14677487020335206
 0.15498458521464936
 0.1631943002259467

In [38]:
import Dragoon.move
args = (1e-3,1e-3,"double")
objFunction = ObjAnalytical

updateHist!(booster,hist,freqs,objFunction)
move(booster,[(1,args[1])])

if args[3] == "double"
    for i in 1:booster.ndisk
        updateHist!(booster,hist,freqs,objFunction)
        move(booster,[(i,-2*args[1])])
        updateHist!(booster,hist,freqs,objFunction)

        g[i] = (hist[2].objvalue-hist[1].objvalue)/(2*args[1])

        if i != booster.ndisk
            move(booster,[(i,args[1]),(i+1,args[1])])
        else
            move(booster,[(i,args[1])])
        end
    end

    for i in 1:booster.ndisk, j in 1:booster.ndisk
        move(booster,[(i,args[2]),(j,args[2])])
        updateHist!(booster,hist,freqs,objFunction)

        move(booster,[(j,-args[2])])
        updateHist!(booster,hist,freqs,objFunction)

        move(booster,[(i,-args[2]),(j,args[2])])
        updateHist!(booster,hist,freqs,objFunction)

        move(booster,[(j,-args[2])])
        updateHist!(booster,hist,freqs,objFunction)
        
        h[i,j] = (hist[4].objvalue-hist[3].objvalue-
                    hist[2].objvalue+hist[1].objvalue)/(args[2]^2)
    end
end

In [47]:
updateHist!(booster,hist,freqs,objFunction; force=true)

In [45]:
Dragoon.shiftdown!(hist)

In [50]:
booster.pos[1] = 1.
hist

10000-element Vector{State}:
 State([1.0, 0.01541943002259467, 0.023629145033892004, 0.03183886004518934, 0.04004857505648668, 0.04825829006778401, 0.05646800507908135, 0.06467772009037868, 0.07288743510167603, 0.08109715011297336, 0.08930686512427068, 0.097516580135568, 0.10572629514686534, 0.11393601015816267, 0.12214572516946, 0.13035544018075734, 0.13856515519205465, 0.14677487020335206, 0.15498458521464936, 0.1631943002259467], -243.30220663023675, Dates.DateTime("2023-06-19T07:16:02.758"))
 State([1.0, 0.01541943002259467, 0.023629145033892004, 0.03183886004518934, 0.04004857505648668, 0.04825829006778401, 0.05646800507908135, 0.06467772009037868, 0.07288743510167603, 0.08109715011297336, 0.08930686512427068, 0.097516580135568, 0.10572629514686534, 0.11393601015816267, 0.12214572516946, 0.13035544018075734, 0.13856515519205465, 0.14677487020335206, 0.15498458521464936, 0.1631943002259467], -243.30220663023263, Dates.DateTime("2023-06-18T22:02:22.758"))
 State([1.0, 0.015419430022

### Optimizer

In [3]:
#=
how to use

linesearch(                     directly modifies booster.pos
    booster,                    
    hist,                       
    freqs,                      frequency range to optimize on
    α,                          base step length e.g. motorspeed*measurementfrequency
    (objective,[options]),      objective function
    (solver,[options]),         provides step direction p
    (derivative,[options]),     provides derivatives for use by solver
    (step,[options]),           calculates steplength
    (search,[options]),         search mode
    (unstuckinator,[options]);  tries to unstuck system from bad local minima
    ϵgrad=0,                    terminate if norm(gradient) falls below this value,
    maxiter=100,                maximum allowed iterations,
    showtrace=false,            wether to print progress,
    showevery=1,                only print every i iterations,
    unstuckisiter=true          wether an unstucking iteration counts towards maxiter
                                    warning! can run endlessly if false
)

returns trace, access with
trace[iteration].x      disc position
                .obj    objective value
                .g      gradient
                .h      hessian
                .t      timestamp
                .T      summed travel time
=#

In [4]:
trace = linesearch(booster,hist,freqs,booster.vmotor*1e-3,
                    ObjAnalytical,
                    SolverNewton("cholesky"),
                    Derivator2(1e-5,1e-6,"double"),
                    StepNorm("unit"),
                    SearchExtendedSteps(2000),
                    UnstuckDont;
                    ϵgrad=0.,maxiter=Int(2e1),showtrace=true);

20-element Vector{Float64}:
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7
 1.216511033151163e7

20×20 Matrix{Float64}:
 -2.43302e14  -2.43302e14  -2.43302e14  …  -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14  …  -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14  …  -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14  -2.43302e14     -2.43302e14  -2.43302e14
 -2.43302e14  -2.43302e14

LinearAlgebra.PosDefException: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [5]:
#obtain result, booster is now in optimized position
#pos2dist(booster.pos)     #in distance space
booster.pos               #in position space

20-element Vector{Float64}:
 0.007209715011297335
 0.01541943002259467
 0.023629145033892004
 0.03183886004518935
 0.040048575056486686
 0.04825829006778402
 0.056468005079081354
 0.06467772009037867
 0.07288743510167602
 0.08109715011297335
 0.08930686512427066
 0.09751658013556799
 0.10572629514686532
 0.11393601015816265
 0.12214572516945998
 0.13035544018075737
 0.13856515519205467
 0.1467748702033521
 0.1549845852146494
 0.16319430022594672

In [6]:
#get handy output
plt = analyse(hist,trace,freqsplot; freqs=freqs,div=10)

#savefig(plt[i],"cool_epic_result_wow.svg")

UndefVarError: UndefVarError: `trace` not defined